In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import date, datetime, timedelta
import warnings
import pytz
import streamlit.components.v1 as components



import pandas as pd
import pymysql
import os
import numpy as np
from openpyxl import load_workbook
import re
import warnings
import httpx
import json
import time
import chess
import chess.pgn
import io
from datetime import datetime
from pytz import timezone
import pytz
from supabase import create_client, Client


from tzlocal import get_localzone  # pip install tzlocal
import time
import chess
import chess.pgn
import io
import pprint
import requests
warnings.filterwarnings('ignore')
import chess_com_download
from streamlit_javascript import st_javascript
from pytz import timezone

from chess_com_download import test_function_test
from chess_com_download import opening_database
from chess_com_download import download_data_chess_com
from chess_com_download import pre_analysis_chessgame
from chess_com_download import analyze_chess_games
from chess_com_download import prepare_game_data
from chess_com_download import insert_to_supabase


from supabase import create_client, Client
from datetime import datetime
import pytz


import streamlit.runtime.app_session as app_session

if not hasattr(app_session.AppSession, "_scriptrunner"):
    app_session.AppSession._scriptrunner = None



In [2]:
# Try to detect local timezone and map it to a valid pytz zone
try:
    import tzlocal  # pip install tzlocal
    local_tz_name = tzlocal.get_localzone_name()  # e.g., 'Asia/Jakarta'
except:
    local_tz_name = "UTC"  # fallback

# If session_state doesn't have timezone, set it to detected
if "user_timezone" not in st.session_state:
    st.session_state.user_timezone = local_tz_name

# Get current time in user's timezone
user_timezone = pytz.timezone(st.session_state.user_timezone)



df_chess_game = download_data_chess_com('jay_fh',5,5,2025)
display (df_chess_game.head() )




2025-08-30 11:14:54.924 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


https://api.chess.com/pub/player/jay_fh/games/2025/05


,pgn,game_class,game_id,created_date,opening,white_username,black_username,game_result,start_time,end_time,termination,white_elo,black_elo,TimeControl
0,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/137988012026,2025.05.01,"0 C00 Name: ECO, dtype: object",mudstorm22,jay_fh,1-0,14:31:51,14:36:08,mudstorm22 won by resignation,1699,1705,180+2
1,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/137992725060,2025.05.01,"0 A84 Name: ECO, dtype: object",jay_fh,michaelsmith7113,0-1,16:53:19,16:57:25,michaelsmith7113 won by checkmate,1699,1804,180+2
2,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138007077278,2025.05.02,"0 C01 Name: ECO, dtype: object",nah-sik,jay_fh,1-0,02:32:01,02:35:31,nah-sik won by resignation,1709,1691,180+2
3,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138007255362,2025.05.02,"0 D11 Name: ECO, dtype: object",jay_fh,mbecker66,0-1,02:42:13,02:48:34,mbecker66 won by resignation,1683,1715,180+2
4,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138009466396,2025.05.02,"0 D94 Name: ECO, dtype: object",jay_fh,chicho18081962,0-1,04:49:55,04:55:32,chicho18081962 won - game abandoned,1676,1736,180+2


In [3]:

df = pre_analysis_chessgame(df_chess_game, 'jay_fh',user_timezone)
display (df.head())

# timecontrol_counts = df['TimeControl'].value_counts()
# popular_timecontrols = timecontrol_counts[timecontrol_counts > 50].index
# print (popular_timecontrols)
# df_final = df[df['TimeControl'].isin(popular_timecontrols)]
# display (df_final.head())
# df['time_of_day'].value_counts() 

,pgn,game_class,game_id,created_date,opening,white_username,black_username,game_result,start_time,end_time,...,opening_name,duration_minutes,hour_of_day,time_of_day,user_side,opponent_username,game_result_user,termination_category,termination_type,simplified_opening
0,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/137988012026,2025-05-01,"0 C00 Name: ECO, dtype: object",mudstorm22,jay_fh,1-0,14:31:51,14:36:08,...,French Defense: Two Knights Variation,4.283333,21,evening 6pm - 6am,black,mudstorm22,lose,lose by resignation,resignation,French Defense
1,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/137992725060,2025-05-01,"0 A84 Name: ECO, dtype: object",jay_fh,michaelsmith7113,0-1,16:53:19,16:57:25,...,Dutch Defense,4.100000,23,NaN,white,michaelsmith7113,lose,lose by checkmate,checkmate,Dutch Defense
2,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138007077278,2025-05-02,"0 C01 Name: ECO, dtype: object",nah-sik,jay_fh,1-0,02:32:01,02:35:31,...,French Defense: Two Knights Variation,3.500000,9,morning 6am - 6 pm,black,nah-sik,lose,lose by resignation,resignation,French Defense
3,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138007255362,2025-05-02,"0 D11 Name: ECO, dtype: object",jay_fh,mbecker66,0-1,02:42:13,02:48:34,...,Benoni Defense: Hawk Variation,6.350000,9,morning 6am - 6 pm,white,mbecker66,lose,lose by resignation,resignation,Benoni Defense
4,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138009466396,2025-05-02,"0 D94 Name: ECO, dtype: object",jay_fh,chicho18081962,0-1,04:49:55,04:55:32,...,Benoni Defense: Hawk Variation,5.616667,11,morning 6am - 6 pm,white,chicho18081962,lose,lose by abandoned,abandoned,Benoni Defense


In [8]:

column = [
'pgn', 'game_class', 'game_id', 'created_datetime','white_username', 'black_username','winner_user', 'winner_side','game_result_user','opponent_elo', 'your_elo'
,'opening_name','simplified_opening','termination_type'
]
display (df[column])

print (df.columns)



,pgn,game_class,game_id,created_datetime,white_username,black_username,winner_user,winner_side,game_result_user,opponent_elo,your_elo,opening_name,simplified_opening,termination_type
0,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/137988012026,2025-05-01 14:31:51,mudstorm22,jay_fh,mudstorm22,empty,lose,1699,1705,French Defense: Two Knights Variation,French Defense,resignation
1,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/137992725060,2025-05-01 16:53:19,jay_fh,michaelsmith7113,michaelsmith7113,empty,lose,1804,1699,Dutch Defense,Dutch Defense,checkmate
2,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138007077278,2025-05-02 02:32:01,nah-sik,jay_fh,nah-sik,empty,lose,1709,1691,French Defense: Two Knights Variation,French Defense,resignation
3,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138007255362,2025-05-02 02:42:13,jay_fh,mbecker66,mbecker66,empty,lose,1715,1683,Benoni Defense: Hawk Variation,Benoni Defense,resignation
4,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/138009466396,2025-05-02 04:49:55,jay_fh,chicho18081962,chicho18081962,empty,lose,1736,1676,Benoni Defense: Hawk Variation,Benoni Defense,abandoned
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/139071014954,2025-05-31 17:44:04,jay_fh,kprep,jay_fh,empty,win,1652,1734,Queen's Pawn Game,Queen's Pawn Game,resignation
137,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/139071137460,2025-05-31 17:48:04,jay_fh,waelshark,jay_fh,empty,win,1709,1742,Englund Gambit Declined: Reversed French,Englund Gambit Declined,resignation
138,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/139084652922,2025-06-01 04:48:14,TilTheEndOfTime,jay_fh,jay_fh,empty,win,1743,1750,King's Indian Attack: Sicilian Variation,King's Indian Attack,resignation
139,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",blitz,https://www.chess.com/game/live/139084722860,2025-06-01 04:52:20,Piyushsaxena0185,jay_fh,Piyushsaxena0185,empty,lose,1779,1743,French Defense: Two Knights Variation,French Defense,resignation


Index(['pgn', 'game_class', 'game_id', 'created_date', 'opening',
       'white_username', 'black_username', 'game_result', 'start_time',
       'end_time', 'termination', 'white_elo', 'black_elo', 'TimeControl',
       'created_week', 'start_time_dtformat', 'end_time_dtformat',
       'created_datetime', 'local_datetime', 'winner_side', 'winner_user',
       'opponent_elo', 'your_elo', 'elo_category', 'opening_name',
       'duration_minutes', 'hour_of_day', 'time_of_day', 'user_side',
       'opponent_username', 'game_result_user', 'termination_category',
       'termination_type', 'simplified_opening'],
      dtype='object')


In [3]:

def opening_database():

    data_a = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/a.tsv"
    data_b = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/b.tsv"
    data_c = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/c.tsv"
    data_d = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/d.tsv"
    data_e = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/e.tsv"

    df_a = pd.read_csv(data_a, sep="\t")
    df_b = pd.read_csv(data_b, sep="\t")
    df_c = pd.read_csv(data_c, sep="\t")
    df_d = pd.read_csv(data_d, sep="\t")
    df_e = pd.read_csv(data_e, sep="\t")

    # Combine the DataFrames
    chess_opening_database = pd.concat([df_a, df_b, df_c, df_d, df_e], ignore_index=True)
    chess_moves = []
    # Iterate over the rows of the combined DataFrame
    for index, row in chess_opening_database.iterrows():
        pgn_string = row['pgn']
        moves = extract_moves_from_pgn(pgn_string)
        chess_moves.append(moves)

        # Add the new column of opening_moves name to the DataFrame
    chess_opening_database['opening_moves'] = chess_moves

    # chess_database_opening_df.head()
    return chess_opening_database

def extract_moves_from_pgn(pgn_string):
    pgn_io = io.StringIO(pgn_string)
    game = chess.pgn.read_game(pgn_io)
    board = game.board()
    moves = []
    for move in game.mainline_moves():
        moves.append(board.san(move))
        board.push(move)
    return moves


chess_database_opening_df = opening_database()

In [4]:
chess_database_opening_df

,eco,name,pgn,opening_moves
0,A00,Amar Opening,1. Nh3,[Nh3]
1,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4,"[Nh3, d5, g3, e5, f4]"
2,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...,"[Nh3, d5, g3, e5, f4, Bxh3, Bxh3, exf4, O-O, f..."
3,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6,"[e3, e5, c4, d6, Nc3, Nc6, b3, Nf6]"
4,A00,Anderssen's Opening,1. a3,[a3]
...,...,...,...,...
3512,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,"[d4, Nf6, c4, g6, Nc3, Bg7, e4, d6, Nf3, O-O, ..."
3513,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,"[d4, Nf6, c4, g6, Nc3, Bg7, e4, d6, Nf3, O-O, ..."
3514,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,"[d4, Nf6, c4, g6, Nc3, Bg7, e4, d6, Nf3, O-O, ..."
3515,E99,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,"[d4, Nf6, c4, g6, Nc3, Bg7, e4, d6, Nf3, O-O, ..."
